In [ ]:
FileName = '/Users/seetha/Desktop/Habenula_Variation/Data/Habenula_AF4_Blue_Redx3/Tiff/Registered/Sorted/Fish1253/dHb/'

# PCA parameters 
pca_components = 4  # Number of pca components to detect from files
num_pca_colors = 50  # Number of colors on the pca maps
num_samples = 10000  # number of random samples to select to do PCA reconstruction
thresh_pca = 0.001  # Threshold above which to plot the pca components
color_map = "polar"

stimulus_on_time = [46, 86, 126, 166, 206, 246]
stimulus_off_time = [65, 106, 146, 186, 226, 266]
color_mat = ['#00FFFF', '#FF0000', '#0000FF', '#FF1493', '#3090C7', '#800000']
time_baseline = [10, 30]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
sns.set_context('notebook');

In [ ]:
from thunder import ThunderContext
print 'Starting Thunder Now. Check console for details'
tsc = ThunderContext.start(appName="thunderpca")

In [ ]:
from thunder import Colorize
image = Colorize.image

In [ ]:
def plot_vertical_lines_onset(stimulus_on_time):
    for ii in xrange(0, np.size(stimulus_on_time)):
        plt.axvline(x=stimulus_on_time[ii], linestyle='-', color='k', linewidth=1)
def plot_vertical_lines_offset(stimulus_off_time):
    for ii in xrange(0, np.size(stimulus_off_time)):
        plt.axvline(x=stimulus_off_time[ii], linestyle='--', color='k', linewidth=1)

In [ ]:
data = tsc.loadImages(FileName, inputFormat='tif')
data = data.medianFilter(size=4)
data = data.toTimeSeries() #detrend(method='linear',order=10)

In [ ]:
data.cache()
A1 = data.pack()
print np.shape(A1)
image(np.mean(A1,0))

In [ ]:
stdMap = data.seriesStdev().pack()

In [ ]:
image(stdMap>4)

In [ ]:
np.where(np.isinf(A1))

In [ ]:
A11 = np.mean(np.reshape(A1, (np.size(A1,0), np.size(A1,1)*np.size(A1,2))),1, dtype=np.float64)
print A1.dtype
print np.where(np.isinf(A11))
print np.where(np.isnan(A11))
print np.max(A11)
print np.shape(A11)
plt.plot(A11);

In [ ]:
from numpy import std
data_filtered = data.filterOnValues(lambda x: std(x) > 4)

data_filtered = tsc.loadSeries(FileName, inputFormat='text', 
                               nkeys=3).toTimeSeries().detrend(method='linear', order=10)

In [ ]:
zscore_data = data.squelch(100).zscore(axis=1, baseline=time_baseline)
zscore_data.cache()
# zscore_data.dims

In [ ]:
A3 = zscore_data.pack()
image(np.mean(A3,0))

In [ ]:
examples = zscore_data.subset(nsamples=200, thresh=3)
with sns.axes_style('darkgrid'):
    plt.plot(examples.T);
    plt.plot(np.mean(examples,0), 'k', linewidth=2)
    plot_vertical_lines_onset(stimulus_on_time)
    plot_vertical_lines_offset(stimulus_off_time)

In [ ]:
Max = zscore_data.max()
Mean = zscore_data.mean()
Min = zscore_data.min()

In [ ]:
print np.shape(Max)
sum(Max)

In [ ]:
with sns.axes_style('darkgrid'):
    plt.plot(Max, label='maximum');
    plt.plot(Mean, label='mean');
    plt.plot(Min, label='minimum');
    plot_vertical_lines_onset(stimulus_on_time)
    plot_vertical_lines_offset(stimulus_off_time)
    plt.legend()

In [ ]:
from thunder import PCA
required_pcs = [0, 1, 2]

model = PCA(k=3).fit(zscore_data)
imgs = model.scores.pack()
reference = data.seriesMean().pack()
maps = Colorize(cmap='polar', scale=100).transform(imgs, background=reference, mixing=0.5)


with sns.axes_style('dark'):
    plt.plot(model.comps.T);
    plot_vertical_lines_onset(stimulus_on_time)
    plot_vertical_lines_offset(stimulus_off_time)
    sns.axlabel("Time (seconds)", "a.u")
    A = []
    for ii in xrange(0, np.size(model.comps.T, 1)):
        A = np.append(A, [str(ii)])
    plt.legend(A, loc=4)
    plt.axhline(y=0, linestyle='-', color='k', linewidth=1)

In [ ]:
def plot_stimulus_in_3d(ax1, pca_components, stimulus_on_time, stimulus_off_time, color_mat,
                        required_pcs, z_direction):
    ## Plot Baseline at beginning
    ax1.plot(pca_components[0:stimulus_on_time[0], required_pcs[0]], 
             pca_components[0:stimulus_on_time[0], required_pcs[1]], 
             pca_components[0:stimulus_on_time[0], required_pcs[2]], zdir=z_direction, color='#808080', linewidth=3)

    print np.shape(pca_components)

    # Plot light on times
    for ii in xrange(0, np.size(stimulus_on_time)):
        ax1.plot(pca_components[stimulus_on_time[ii]:stimulus_off_time[ii], required_pcs[0]], 
                 pca_components[stimulus_on_time[ii]:stimulus_off_time[ii], required_pcs[1]], 
                 pca_components[stimulus_on_time[ii]:stimulus_off_time[ii], required_pcs[2]], zdir=z_direction,
                 color=color_mat[ii], linewidth=3)

    # Plot light off times
    for ii in xrange(0, np.size(stimulus_on_time)):
        if ii == np.size(stimulus_on_time) - 1:
            #            print ii
            ax1.plot(pca_components[stimulus_off_time[ii]:stimulus_off_time[ii] + 20, required_pcs[0]], 
                     pca_components[stimulus_off_time[ii]:stimulus_off_time[ii] + 20, required_pcs[1]], 
                     pca_components[stimulus_off_time[ii]:stimulus_off_time[ii] + 20, required_pcs[2]],
                     zdir=z_direction, color=color_mat[ii], linewidth=2, linestyle='--')
        else:

            ax1.plot(pca_components[stimulus_off_time[ii]:stimulus_on_time[ii + 1], required_pcs[0]], 
                     pca_components[stimulus_off_time[ii]:stimulus_on_time[ii + 1], required_pcs[1]], 
                     pca_components[stimulus_off_time[ii]:stimulus_on_time[ii + 1], required_pcs[2]], zdir=z_direction,
                     color=color_mat[ii], linewidth=2, linestyle='--')

    ## Plot Baseline at end of stimulus
    ax1.plot(pca_components[stimulus_off_time[ii] + 20:, required_pcs[0]], 
             pca_components[stimulus_off_time[ii] + 20:, required_pcs[1]], 
             pca_components[stimulus_off_time[ii] + 20:, required_pcs[2]], zdir=z_direction, color='#000000',
             linewidth=3)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
pca_components = model.comps.T
with sns.axes_style('dark'):
    fig1 = plt.figure()
    ax1 = fig1.add_subplot(111, projection='3d')
    plot_stimulus_in_3d(ax1, pca_components, stimulus_on_time, stimulus_off_time,color_mat,
                        required_pcs, 'y')

In [ ]:
from numpy import amax
image(maps, clim=(-0.01, 0.01))

In [ ]:
from numpy import asarray
from numpy import newaxis, squeeze
pts = model.scores.subset(10, thresh=0.05, stat='norm')
recon = asarray(map(lambda x: (x[0] * model.comps[0, :] + x[1] * model.comps[1, :]).tolist(), pts))
clrs = Colorize(cmap='polar', scale=100).transform([pts[:,0][:,newaxis], pts[:,1][:,newaxis]]).squeeze()
with sns.axes_style('dark'):
    fs = plt.figure(figsize=(15,10))
    plt.gca().set_color_cycle(clrs)
    plt.plot(recon.T);
    plot_vertical_lines_onset(stimulus_on_time)
    plot_vertical_lines_offset(stimulus_off_time)
    plt.axhline(y=0, linestyle='-', color='k', linewidth=1)